In [1]:
import pandas as pd
from pandas import DataFrame, read_csv
import networkx as nx

KeyboardInterrupt: 

In [ ]:
# with open('atari_graphs/atari_edges_fixed.csv', 'a', encoding="utf8") as file_1, open('atari_graphs/atari_edges.csv', "r", encoding="utf8") as file_2:
#     for line in file_2:
#         if ',quotes' not in line:
#             file_1.write(line)

In [ ]:
df = pd.read_csv('atari_edges_fixed.csv', header = None)
df.columns = ['src', 'dst', 'tema', 'type']
df.head()

In [ ]:
df_simple = df[['src', 'tema']]

In [ ]:
len(pd.unique(df_simple['src']))

In [ ]:
df = df_simple.groupby(df_simple.columns.tolist()).size().reset_index().rename(columns={0:'weight'})


In [ ]:
df.head()

In [ ]:
df['src_original'] = df['src']
df['src'] = pd.Categorical(df['src'])
df['src'] = df['src'].cat.codes
df['tema_original'] = df['tema']
df['tema'] = pd.Categorical(df['tema'])
df['tema'] = df['tema'].cat.codes
df['tema'] = df['tema'] + 200000

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
G = nx.Graph()
for index, row in df.iterrows():
    
    G.add_node(row['src'], bipartite=0)
    G.add_node(row['tema'], bipartite=1)
    
    if G.has_edge(row['src'], row['tema']):
                        G[row['src']][row['tema']]['weight'] += row['weight']
    else:
        G.add_edge(row['src'], row['tema'], weight = row['weight'])

In [ ]:
len(pd.unique(df['src']))

In [ ]:
nx.is_bipartite(G)

In [ ]:
# nx.write_graphml_lxml(G, "atari_graphs/user_topics_AA.graphml")

In [ ]:
top_nodes = {n for n, d in G.nodes(data=True) if d['bipartite']==1}
bottom_nodes = set(G) - top_nodes

In [ ]:
# def forum_projection(B, nodes, other_nodes):
#     lengths = {n: len(B[n]) for n in other_nodes}
#     if B.is_directed():
#         pred = B.pred
#         G = nx.DiGraph()
#     else:
#         pred = B.adj
#         G = nx.Graph()
#     G.graph.update(B.graph)
#     G.add_nodes_from((n, B.nodes[n]) for n in nodes)
#     for u in nodes:
#         unbrs = set(B[u]) #nodos a los que conecta del otro nivel
#         nbrs2 = {n for nbr in unbrs for n in B[nbr] if n != u} #nodos con los que coincide
#         for v in nbrs2:
#             vnbrs = set(pred[v])
#             common_degree = ((len(B[n]), B[u][n]['weight'] + B[v][n]['weight']) for n in unbrs & vnbrs)
#             weight = sum(wt / (deg - 1) for deg, wt in common_degree if deg > 1)
#             G.add_edge(u, v, weight=weight)
#     return G

In [ ]:
def forum_projection_to_file(B, nodes, other_nodes, filename):
    lengths = {n: len(B[n]) for n in other_nodes}
    if B.is_directed():
        pred = B.pred
    else:
        pred = B.adj
    vnbrs_sets = {n: set(pred[n]) for n in nodes}

    i = 0
    print("starting...")
    with open('{}.csv'.format(filename), 'w', encoding="utf8") as file:
        for u in nodes:
            i = i + 1
            if i % 100 == 0:
                print("#node:", i)
            unbrs = set(B[u]) #nodos a los que conecta del otro nivel
            nbrs2 = {n for nbr in unbrs for n in B[nbr] if n != u} #nodos con los que coincide
            for v in nbrs2:
                vnbrs = vnbrs_sets[v]
                common_degree = ((lengths[n], B[u][n]['weight'] + B[v][n]['weight']) for n in unbrs & vnbrs)
                weight = sum(wt / (deg - 1) for deg, wt in common_degree if deg > 1)
                file.write("{},{},{:.8f}\n".format(u,v,weight))
                del common_degree
                del weight
            del unbrs
            del nbrs2
    print("done.")

In [ ]:
G_top = forum_projection_to_file(G, top_nodes, bottom_nodes, "atari_edges_flat_top")

In [26]:
G_bottom = forum_projection_to_file(G, bottom_nodes, top_nodes, "atari_edges_flat_bottom")

starting...
#node: 100
#node: 200
#node: 300
#node: 400
#node: 500
#node: 600
#node: 700
#node: 800
#node: 900
#node: 1000
#node: 1100
#node: 1200
#node: 1300
#node: 1400
#node: 1500
#node: 1600
#node: 1700
#node: 1800
#node: 1900
#node: 2000
#node: 2100
#node: 2200
#node: 2300
#node: 2400
#node: 2500
#node: 2600
#node: 2700
#node: 2800
#node: 2900
#node: 3000
#node: 3100
#node: 3200
#node: 3300
#node: 3400
#node: 3500
#node: 3600
#node: 3700
#node: 3800
#node: 3900
#node: 4000
#node: 4100
#node: 4200
#node: 4300
#node: 4400
#node: 4500
#node: 4600
#node: 4700
#node: 4800
#node: 4900
#node: 5000
#node: 5100
#node: 5200
#node: 5300
#node: 5400
#node: 5500
#node: 5600
#node: 5700
#node: 5800
#node: 5900
#node: 6000
#node: 6100
#node: 6200
#node: 6300
#node: 6400
#node: 6500
#node: 6600
#node: 6700
#node: 6800
#node: 6900
#node: 7000
#node: 7100
#node: 7200
#node: 7300
#node: 7400
#node: 7500
#node: 7600
#node: 7700
#node: 7800
#node: 7900
#node: 8000
#node: 8100
#node: 8200
#node: 8300
#

In [27]:
H =nx.read_weighted_edgelist("atari_edges_flat_bottom.csv", delimiter =",")

KeyboardInterrupt: 

In [28]:
H = nx.read_weighted_edgelist("atari_edges_flat_bottom_filtered.csv", delimiter =",")

In [32]:
len(list(H.nodes()))
# Estaban DUPLICADAS en el fichero en ambas direcciones, aquí las pisa.

17258

In [31]:
import community # al igual hace falta instalar pip install python-louvain
partition = community.best_partition(H)

KeyboardInterrupt: 

In [ ]:
df['community'] = df.apply(lambda row: partition[row.Source], axis=1)

In [ ]:
df.to_csv('communities_AA_tab.csv', sep ='\t')

In [ ]:
df.groupby(['community']).agg({'weight': 'sum'}).apply(lambda x:
                                                 100 * x / float(x.sum()))

In [ ]:
pesos = df.groupby(['community', 'topic_id']).agg({'weight': 'sum'})
pesos['percentage'] = pesos['weight']/pesos.groupby('community')['weight'].transform('sum')

In [ ]:
pesos